In [1]:
import keras
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [3]:
np.arange(4)

array([0, 1, 2, 3])

In [4]:
(x_train, y_train),(x_valid, y_valid) = mnist.load_data()

In [5]:
y_train = keras.utils.to_categorical(y_train, 10)
y_valid = keras.utils.to_categorical(y_valid, 10)

In [6]:
y_valid.shape

(10000, 10)

In [7]:
from sklearn.model_selection import train_test_split
x_t, x_v, y_t, y_v = x_train, x_valid, y_train, y_valid
#x_t, x_v, y_t, y_v = train_test_split(x_valid, y_valid, test_size=0.2, random_state=42) 

In [8]:
print(x_t.shape, y_t.shape, x_v.shape, y_v.shape)

(60000, 28, 28) (60000, 10) (10000, 28, 28) (10000, 10)


In [9]:
x_t = x_t.astype('float32')
x_v = x_v.astype('float32')
x_t /= 255
x_v /= 255

In [10]:
x_t = x_t.reshape(x_t.shape[0], 784)
x_v = x_v.reshape(x_v.shape[0], 784)

In [11]:
def sigmoid(z):
    return (1 / (1 + np.exp(-z)))

def sigmoid_(z):
    return (sigmoid(z)*(1-sigmoid(z)))


In [12]:
class Neuralnet(object):
    def __init__(self, size):
        self.layers = len(size)
        self.size = size
        self.weights = [np.random.randn(y, x) for x, y in zip(size[:-1], size[1:])]
        self.biases = [np.random.randn(y, 1) for y in size[1:]]
        
    def forward(self, a):
        a = a.T
        for w, b in zip(self.weights, self.biases):
            a = sigmoid((w @ a) + b)
        return a.T
    
    def SGD(self, X, y, epochs, lr, bs, test_data = False):
        
        if test_data: 
            X_t, y_t = test_data
            n_test = len(X_t)
        n = len(X)
        for e in range(epochs):
            X, y = shuffle(X, y, random_state=42)
            X_batches = [X[k:k+bs] for k in range(0, n, bs)]
            y_batches = [y[k:k+bs] for k in range(0, n, bs)]
            for x_, y_ in zip(X_batches, y_batches):
                self.update_minibatch(lr/bs, x_, y_)
            
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(e, self.evaluate(X_t, y_t), n_test))
            else:
                print ("Epoch {} complete".format(e))
                
    def update_minibatch(self,lr, x, y):
        dW, db = self.backprop(x,y)
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] - lr * dW[i]
            self.biases[i] = self.biases[i] - lr * db[i]
            
            
        
    def backprop(self,x,y):
        dw = [np.zeros(w.shape) for w in self.weights]
        db = [np.zeros(b.shape) for b in self.biases]
        a = x.T
        activations = [a]
        zs = []
        for w, b in zip(self.weights, self.biases):
            z = w @ a + b
            zs.append(z)
            a = sigmoid(z)
            activations.append(a)
        delta = (a - y.T) * sigmoid_(z)
        #print(delta.shape)
        dw[-1] = delta @ activations[-2].T
        db[-1] = np.reshape(np.average(delta, axis=1),(db[-1].shape[0],1))
        
        for l in range(2, self.layers):
            delta = self.weights[-l+1].T @ delta * sigmoid_(zs[-l])
            #print(delta.shape)
            dw[-l] = delta @ activations[-l-1].T
            db[-l] = np.reshape(np.average(delta, axis=1),(db[-l].shape[0],1))
        
        return dw, db
        
        
    def evaluate(self, x, y):
        y_ = [np.argmax(a) for a in self.forward(x)]
        y = [np.argmax(l) for l in y]
        return sum(int(a==l) for a,l in zip(y_, y))

In [13]:
net = Neuralnet([784, 16, 16, 10])

In [14]:
net.SGD(x_t, y_t, 30, 3.0, 16, test_data=(x_v, y_v))

Epoch 0: 8884 / 10000
Epoch 1: 9132 / 10000
Epoch 2: 9177 / 10000
Epoch 3: 9230 / 10000
Epoch 4: 9240 / 10000
Epoch 5: 9242 / 10000
Epoch 6: 9298 / 10000
Epoch 7: 9310 / 10000
Epoch 8: 9344 / 10000
Epoch 9: 9324 / 10000
Epoch 10: 9343 / 10000
Epoch 11: 9293 / 10000
Epoch 12: 9356 / 10000
Epoch 13: 9312 / 10000
Epoch 14: 9385 / 10000
Epoch 15: 9383 / 10000
Epoch 16: 9389 / 10000
Epoch 17: 9378 / 10000
Epoch 18: 9396 / 10000
Epoch 19: 9403 / 10000
Epoch 20: 9375 / 10000
Epoch 21: 9418 / 10000
Epoch 22: 9423 / 10000
Epoch 23: 9350 / 10000
Epoch 24: 9401 / 10000
Epoch 25: 9380 / 10000
Epoch 26: 9423 / 10000
Epoch 27: 9417 / 10000
Epoch 28: 9420 / 10000
Epoch 29: 9422 / 10000


# Single fwd-pass and backprop for one batch

In [15]:
bs = 32 #batch size

In [16]:
x = x_t[:bs].T; x.shape # take batch and transpose of ease of calculations - Features

(784, 32)

In [17]:
y = y_t[:bs].T; y.shape # take batch and transpose of ease of calculations - Labels

(10, 32)

In [18]:
w1 = np.random.randn(128, 784) # Initialise weights for 1st layer-128 number of neurons, -784 features incoming
b1 = np.zeros((128,1))         # Initialise biases for 1st layer  -128 number of neurons
w2 = np.random.randn(10, 128)  # Initialise weights for 2nd layer -10 number of neurons, -128 features incoming
b2 = np.zeros((10,1))          # Initialise biases for 2nd layer -10 number of neurons

## Learning loop

### Forward pass

In [19]:
z1 = w1 @ x + b1; z1.shape # Vectorize calculations for 1st layer

(128, 32)

In [20]:
a1 = sigmoid(z1); a1.shape # Adding the non-linearity - sigmoid activation

(128, 32)

In [21]:
z2 = w2 @ a1 + b2; z2.shape # Vectorize calculations for 2nd/output layer

(10, 32)

In [22]:
a2 = sigmoid(z2); a2.shape # Adding the non-linearity - sigmoid activation

(10, 32)

### Quadratic cost

In [23]:
c = (a2 - y); c.shape # Quadratic cost derivative

(10, 32)

In [24]:
zp2 = sigmoid_(z2); zp2.shape # Sigmoid prime for output activations

(10, 32)

### Backprop

In [25]:
delta2 = c * zp2; delta2.shape # Error for last layer

(10, 32)

In [26]:
db2 = np.reshape(np.average(delta2, axis=1),(10,1)); db2.shape # Derivative of cost w.r.t biases 2nd/output layer

(10, 1)

In [27]:
dw2 = delta2 @ a1.T; dw2.shape # Derivative of cost w.r.t weights 2nd/output layer

(10, 128)

In [28]:
delta1 = (w2.T @ delta2) * sigmoid_(z1); delta1.shape # Cost/Error w.r.t 1st layer

(128, 32)

In [30]:
db1 = np.reshape(np.average(delta1, axis=1),(128,1)); db1.shape # Derivative of cost w.r.t biases 1st hidden layer

(128, 1)

In [31]:
dw1 = delta1 @ x.T; dw1.shape # # Derivative of cost w.r.t weights for 1st hidden layer  layer

(128, 784)

### Update weights

In [ ]:
w1 = w1 - 0.01*dw1/bs
b1 = b1 - 0.01*db1/bs
w2 = w2 - 0.01*dw2/bs
b2 = b2 - 0.01*db2/bs